In [83]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [84]:
from nba_api.stats.endpoints import leaguegamefinder
szn_20_21 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2020-12-22",
                                                date_to_nullable="2021-7-22", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
szn_21_22 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2021-10-19",
                                              date_to_nullable="2022-7-19",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_22_23 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2022-10-18",
                                              date_to_nullable="2023-7-12",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_23_24 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2023-10-24",
                                              date_to_nullable="2024-4-14",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_24_25 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2024-10-18",
                                                date_to_nullable="2025-4-18", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
szn_20_21 = szn_20_21.get_data_frames()[0]
szn_20_21 = pd.DataFrame(szn_20_21)
gameid = szn_20_21['GAME_ID'].unique()
szn_21_22 = szn_21_22.get_data_frames()[0]
szn_21_22 = pd.DataFrame(szn_21_22)
szn_22_23 = szn_22_23.get_data_frames()[0]
szn_22_23 = pd.DataFrame(szn_22_23)
szn_23_24 = szn_23_24.get_data_frames()[0]
szn_23_24 = pd.DataFrame(szn_23_24)
szn_24_25 = szn_24_25.get_data_frames()[0]
szn_24_25 = pd.DataFrame(szn_24_25)
ids = list(szn_20_21['TEAM_ID'].unique())

In [85]:
lst=[szn_20_21, szn_21_22, szn_22_23, szn_23_24, szn_24_25]
rest_list = []
for x in lst:
    x["TotalFGMade"] = x.groupby('GAME_ID')['FGM'].transform('sum')
    x["TotalFGAtt"] = x.groupby('GAME_ID')['FGA'].transform('sum')
    x['Total3PM'] = x.groupby('GAME_ID')['FG3M'].transform('sum')
    x["TotalTOV"] = x.groupby('GAME_ID')['TOV'].transform('sum')
    x["TotalFTA"] = x.groupby('GAME_ID')['FTA'].transform('sum')
    x['TotalOREB'] = x.groupby('GAME_ID')['OREB'].transform('sum')
    x['TotalGamePoints'] = x.groupby('GAME_ID')['PTS'].transform('sum')
    x['OppOREB'] = x['TotalOREB'] - x['OREB']
    x['TDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
    x['TotalGameFTM'] = x.groupby('GAME_ID')['FTM'].transform('sum')
    x['TotalDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
    x['TotalAST'] = x.groupby('GAME_ID')['AST'].transform('sum')
    x['TotalSTL'] = x.groupby('GAME_ID')['STL'].transform('sum')
    x['TotalBLK'] = x.groupby('GAME_ID')['BLK'].transform('sum')
    x['TotalPF'] = x.groupby('GAME_ID')['PF'].transform('sum')
    x['TotalTOV'] = x.groupby('GAME_ID')['TOV'].transform('sum')
    x['GAME_DATE'] = pd.to_datetime(x['GAME_DATE'])
    copy = x.sort_values(by=['TEAM_ID', 'GAME_DATE'])
    copy = copy[['GAME_DATE','TEAM_ID', 'GAME_ID', 'TEAM_NAME']]
    copy['REST DAYS'] = copy.groupby('TEAM_ID')['GAME_DATE'].diff().dt.days
    copy['REST DAYS'] = copy['REST DAYS'] - 1
    index = copy.set_index(['GAME_ID', 'TEAM_ID'])
    rest_list.append(index['REST DAYS'].to_dict())

In [86]:
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 3)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return float(x['OREB']) / (float(x['OREB'] + x['OPPDREB']))
def FTRATE(x):
    return float(x['FTA']) / float(x['FGA'])

In [87]:
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 3)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 3)
def DREBP(x):
    return float(x['DREB']) / float(x['DREB'] + x['OppOREB'])
def OppFTRATE(x):
    return float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA'])
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]
def teamPIE(x):
    teampie = (x['PTS'] + x['FGM'] + x['FTM'] - x['FGA'] - x['FTA'] + x['DREB'] + (0.5 * x['OREB']) + x['AST'] + x['STL'] + (x['BLK']*0.5) - x['PF'] - x['TOV']) 
    gamepie = (x['TotalGamePoints'] + x['TotalFGMade'] + x['TotalGameFTM'] - x['TotalFGAtt'] - x['TotalFTA'] + x['TotalDREB'] + (0.5 * x['TotalOREB']) + x['TotalAST'] + x['TotalSTL'] + (0.5 * x['TotalBLK']) - x['TotalPF'] - x['TotalTOV'])
    return round(float(teampie) / float(gamepie), 4)
def diff(x):
    return abs(x['home_PTS'] - x['away_PTS'])
def away_POSS(x):
    return (x['away_FGA'] + 0.44 * x['away_FTA'] - x['away_OREB'] + x['away_TOV'])
def home_POSS(x):
    return (x['home_FGA'] + 0.44 * x['home_FTA'] - x['home_OREB'] + x['home_TOV'])
def home_PACE(x):
    return (x['home_POSS'] / (x['home_MIN'] / 5)) * 48
def away_PACE(x):
    return (x['away_POSS'] / (x['away_MIN'] / 5)) * 48
def elo_DIFF(x):
    return (x['home_ELO'] - x['away_ELO'])
def pace_DIFF(x):
    return (x['home_PACE'] - x['home_PACE'])
def rest_DIFF(x):
    return (x['home REST DAYS'] - x['away REST DAYS'])
def home_back2back(x):
    if (x['home REST DAYS'] == 0):
        return 1
    return 0
def away_back2back(x):
    if (x['away REST DAYS'] == 0):
        return 1
    return 0
def netRating_DIFF(x):
    return (x["home_NetRating"] - x["away_NetRating"])

In [88]:
for x in lst:
    x['EFG'] = x.apply(EFG, axis=1)
    x['TOVP'] = x.apply(TOVP, axis=1)
    x['OREBP'] = x.apply(OREBP, axis=1)
    x['FTRATE'] = x.apply(FTRATE, axis=1)
    x['OppEFG'] = x.apply(OppEFG, axis=1)
    x['OppTOVP'] = x.apply(OppTOVP, axis=1)
    x['DREBP'] = x.apply(DREBP, axis=1)
    x['OppFTRATE'] = x.apply(OppFTRATE, axis=1)
    x["AWAY"] = x.apply(away, axis=1)
    x["HOME"] = x.apply(home, axis=1)
    x['TEAMPIE'] = x.apply(teamPIE, axis=1)
    x['POSS'] = x['FGA'] + 0.44 * x['FTA'] - x['OREB'] + x['TOV']
    x['OffRating'] = (x['PTS'] * 100) / (x['POSS'] + 1e-6)
    x['OppFGA'] = x['TotalFGAtt'] - x['FGA']
    x['OppFTA'] = x['TotalFTA'] - x['FTA']
    x['OppOREB'] = x['TotalOREB'] - x['OREB']
    x['OppTOV'] = x['TotalTOV'] - x['TOV']
    x['OppPTS'] = x['TotalGamePoints'] - x['PTS']
    x['OppPOSS'] = x['OppFGA'] + 0.44 * x['OppFTA'] - x['OppOREB'] + x['OppTOV']
    x['DefRating'] = (x['OppPTS'] * 100) / (x['OppPOSS'] + 1e-6)
    x['NetRating'] = x['OffRating'] - x['DefRating']


In [89]:
szn_20_21.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)
szn_21_22.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'], inplace=True)
szn_22_23.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)
szn_23_24.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)
szn_24_25.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)

In [90]:
for i, x in enumerate(lst):
    x['_original_index'] = range(len(x))
    x_sorted = x.sort_values(['TEAM_ID', 'SEASON_ID', 'GAME_DATE']).reset_index(drop=True)
    ewma_cols = ['PTS', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                 'OREB', 'DREB', 'AST', 'STL', 'TOV', 'PF', 'PLUS_MINUS', 'EFG', 'TOVP',
                 'OREBP', 'FTRATE', 'OppEFG', 'OppTOVP', 'DREBP', 'OppFTRATE', 'TEAMPIE',
                 'OffRating', 'DefRating', 'NetRating']
    for col in ewma_cols:
        x_sorted[f"EWMA {col}"] = (
            x_sorted.groupby(['TEAM_ID', 'SEASON_ID'])[col]
            .apply(lambda x: x.shift(1).ewm(span=7, min_periods=7).mean())
            .reset_index(level=[0, 1], drop=True)
        )
    x_sorted = x_sorted.sort_values('_original_index').reset_index(drop=True)
    x_sorted = x_sorted.drop(columns=['_original_index'])
    lst[i] = x_sorted

In [91]:
def combine_game_rows(df):
    df = df.copy()
    df['is_home'] = df['MATCHUP'].str.contains('vs.', na=False)
    stat_columns = [col for col in df.columns if col not in 
                   ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 
                    'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'AWAY', 'HOME']]
    home_data = df[df['is_home'] == True].copy()
    away_data = df[df['is_home'] == False].copy()
    home_rename = {col: f'home_{col}' for col in stat_columns}
    home_rename.update({
        'TEAM_ID': 'home_TEAM_ID',
        'TEAM_ABBREVIATION': 'home_TEAM',
        'TEAM_NAME': 'home_TEAM_NAME',
        'WL': 'home_WL'
    })
    home_data = home_data.rename(columns=home_rename)
    away_rename = {col: f'away_{col}' for col in stat_columns}
    away_rename.update({
        'TEAM_ID': 'away_TEAM_ID',
        'TEAM_ABBREVIATION': 'away_TEAM',
        'TEAM_NAME': 'away_TEAM_NAME',
        'WL': 'away_WL'
    })
    away_data = away_data.rename(columns=away_rename)
    merge_cols = ['GAME_ID', 'GAME_DATE']
    if 'SEASON_ID' in df.columns:
        merge_cols.append('SEASON_ID')
    combined = pd.merge(
        home_data[merge_cols + [col for col in home_data.columns if col not in merge_cols]],
        away_data[merge_cols + [col for col in away_data.columns if col not in merge_cols]],
        on=merge_cols,
        how='inner'
    )
    home_ewma_cols = [col for col in combined.columns if 'home_EWMA' in col]
    away_ewma_cols = [col for col in combined.columns if 'away_EWMA' in col]
    for x,y in zip(home_ewma_cols, away_ewma_cols):
        name = x.split(" ")
        combined[f'EWMA {name[-1]} DIFF'] = combined[f'{x}'] - combined[f'{y}']

    combined['home_win'] = (combined['home_WL'] == 'W').astype(int)
    return combined

In [92]:
team_id_dict = dict.fromkeys(ids,1500) 
def ELO(x):
    prob_home = 1 / (1 + 10**((team_id_dict[x['away_TEAM_ID']] - team_id_dict[x['home_TEAM_ID']] - 100)/400))
    prob_away = 1 - prob_home
    if (x['home_win'] == 1):
        pre_elo_home = team_id_dict[x['home_TEAM_ID']]
        team_id_dict[x['home_TEAM_ID']] = pre_elo_home + 20 * (1 - prob_home)
        pre_elo_away = team_id_dict[x['away_TEAM_ID']]
        team_id_dict[x['away_TEAM_ID']] = pre_elo_away + 20 * (0 - prob_away)
    else:
        pre_elo_home = team_id_dict[x['home_TEAM_ID']]
        team_id_dict[x['home_TEAM_ID']] = pre_elo_home + 20 * (0 - prob_home)
        pre_elo_away = team_id_dict[x['away_TEAM_ID']]
        team_id_dict[x['away_TEAM_ID']] = pre_elo_away + 20 * (1 - prob_away)
    return pre_elo_home, pre_elo_away

In [93]:
combined_seasons = []
for index,df in enumerate(lst):
    combined_df = combine_game_rows(df)
    combined_df['DIFF'] = combined_df.apply(diff, axis=1)
    combined_df.drop(columns=['AWAY_y','HOME_y', 'away_is_home','home_is_home','AWAY_x', 'HOME_x',], inplace=True)
    combined_df = combined_df.sort_values(by='GAME_DATE', ascending=True)
    combined_df[['home_ELO', 'away_ELO']] = combined_df.apply(ELO, axis=1, result_type='expand')
    combined_df['away_POSS'] = combined_df.apply(away_POSS,axis=1)
    combined_df['home_POSS'] = combined_df.apply(home_POSS,axis=1)
    combined_df['away_PACE'] = combined_df.apply(away_PACE, axis=1)
    combined_df['home_PACE'] = combined_df.apply(home_PACE, axis=1)
    home_ref = pd.Series(zip(combined_df['GAME_ID'], combined_df['home_TEAM_ID']),index=combined_df.index)
    away_ref = pd.Series(zip(combined_df['GAME_ID'] , combined_df['away_TEAM_ID']),index=combined_df.index)
    combined_df['home REST DAYS'] = pd.Series(home_ref).map(rest_list[index])
    combined_df['away REST DAYS'] = pd.Series(away_ref).map(rest_list[index])
    combined_df['home Back To Back'] = combined_df.apply(home_back2back, axis=1)
    combined_df['away Back To Back'] = combined_df.apply(away_back2back, axis=1)
    combined_df['elo DIFF'] = combined_df.apply(elo_DIFF, axis=1)
    combined_df['rest DIFF'] = combined_df.apply(rest_DIFF, axis=1)
    combined_df['NetRating DIFF'] = combined_df.apply(netRating_DIFF, axis=1)
    combined_seasons.append(combined_df)
    team_id_dict.update((x, 1500 + ((y-1500) * (1 - (1/3)))) for x, y in team_id_dict.items())
szn_20_21 = combined_seasons[0]
szn_21_22 = combined_seasons[1]
szn_22_23 = combined_seasons[2]
szn_23_24 = combined_seasons[3]
szn_24_25 = combined_seasons[4]

In [95]:
elo_df = pd.DataFrame(list(team_id_dict.items()), columns=['team_ID', 'elo'])
elo_df.to_sql("2024-2025 Final ELO Rankings", conn, if_exists="replace", index=False)

30

In [98]:
print(list(szn_20_21.columns))

['GAME_ID', 'GAME_DATE', 'SEASON_ID', 'home_TEAM_ID', 'home_TEAM', 'home_TEAM_NAME', 'MATCHUP_x', 'home_WL', 'home_MIN', 'home_PTS', 'home_FGM', 'home_FGA', 'home_FG_PCT', 'home_FG3M', 'home_FG3A', 'home_FG3_PCT', 'home_FTM', 'home_FTA', 'home_FT_PCT', 'home_OREB', 'home_DREB', 'home_REB', 'home_AST', 'home_STL', 'home_BLK', 'home_TOV', 'home_PF', 'home_PLUS_MINUS', 'home_TotalGamePoints', 'home_TotalGameFTM', 'home_TotalDREB', 'home_TotalAST', 'home_TotalSTL', 'home_TotalBLK', 'home_TotalPF', 'home_EFG', 'home_TOVP', 'home_OREBP', 'home_FTRATE', 'home_OppEFG', 'home_OppTOVP', 'home_DREBP', 'home_OppFTRATE', 'home_TEAMPIE', 'home_POSS', 'home_OffRating', 'home_OppFGA', 'home_OppFTA', 'home_OppTOV', 'home_OppPTS', 'home_OppPOSS', 'home_DefRating', 'home_NetRating', 'home_EWMA PTS', 'home_EWMA FGA', 'home_EWMA FG_PCT', 'home_EWMA FG3M', 'home_EWMA FG3A', 'home_EWMA FG3_PCT', 'home_EWMA FTM', 'home_EWMA FTA', 'home_EWMA FT_PCT', 'home_EWMA OREB', 'home_EWMA DREB', 'home_EWMA AST', 'home_E

In [96]:
szn_20_21.to_sql("2020-2021 Team Data Combined", conn, if_exists="replace", index=False)
szn_21_22.to_sql("2021-2022 Team Data Combined", conn, if_exists="replace", index=False)
szn_22_23.to_sql("2022-2023 Team Data Combined", conn, if_exists="replace", index=False)
szn_23_24.to_sql("2023-2024 Team Data Combined", conn, if_exists="replace", index=False)
szn_24_25.to_sql("2024-2025 Team Data Combined", conn, if_exists="replace", index=False)

1225